# Comparing the Virtual Product output with notebook output

Need to test if virtual products is behaving as expected before attempting to scale the analysis using datacube stats.
To generate the virtual product, run `virtual_product_test.py`.

**NOTE:**

The results show that the VP version is underestimating seasonal NDVI climatologies.  Appears as though the `group_by: all-time` parameter (in the yaml) is messing with the results as I suspect it runs before our function (and our function does its own, bespoke 'groupby' so it doesn't require VP to do its own).  I think what we really want is for datacube-virtual to have an option like `group_by: None`, in which it will skip this step.


In [ ]:
from datacube import Datacube
import numpy as np
import xarray as xr
from dask.distributed import Client
import matplotlib.pyplot as plt
import sys

sys.path.append('../../Scripts')
from dea_datahandling import load_ard

In [ ]:
client = locals().get('client', None)
if client is not None:
    client.close()
    del client
    
client = Client(n_workers=4, threads_per_worker=1, memory_limit='7GB')
client


## Load data matching extents of VP 

In [ ]:
lat, lon = -33.2, 149.1
buffer = 0.05
time = ('1987', '2010')

In [ ]:
dc = Datacube(env='c3-samples')

query = {'lon': (lon - buffer, lon + buffer),
         'lat': (lat - buffer, lat + buffer),
         'time': time,
         'measurements':['nbart_red', 'nbart_nir', 'fmask'],
         'output_crs' :'EPSG:3577',
         'resolution' : (-30, 30),
         'resampling' :{"fmask": "nearest", "*": "average"}}    

In [ ]:
ds = load_ard(dc=dc, dask_chunks={'x':100, 'y':100}, lazy_load=True,
            products=['ga_ls5t_ard_3', 'ga_ls7e_ard_3'],
            **query)
print(ds)

## Calculate climatologies

In [ ]:
quarter= {'JFM': [1,2,3],
                       'FMA': [2,3,4],
                       'MAM': [3,4,5],
                       'AMJ': [4,5,6],
                       'MJJ': [5,6,7],
                       'JJA': [6,7,8],
                       'JAS': [7,8,9],
                       'ASO': [8,9,10],
                       'SON': [9,10,11],
                       'OND': [10,11,12],
                       'NDJ': [11,12,1],
                       'DJF': [12,1,2],
                      }

def ndvi_clim_mean(data):
    print(data)
    def attrs_reassign(da, dtype=np.float32):
        da_attr = data.attrs
        da = da.assign_attrs(**da_attr)
        return da
    
    ndvi = xr.Dataset(data_vars={'ndvi': (data.nbart_nir - data.nbart_red) / (data.nbart_nir + data.nbart_red)},
                          coords=data.coords,
                          attrs=dict(crs=data.crs))
    print(ndvi)
    ndvi_var = []
    for q in quarter:
        ix=ndvi['time.month'].isin(quarter[q])
        ndvi_clim_mean=ndvi.where(ix,drop = True).mean(dim='time')   
        ndvi_clim_mean=ndvi_clim_mean.to_array(name='ndvi_clim_mean_'+q).drop('variable').squeeze()
        ndvi_var.append(ndvi_clim_mean)

    q_clim_mean = xr.merge(ndvi_var)   

    #assign back attributes
    q_clim_mean.attrs = data.attrs 
    q_clim_mean = q_clim_mean.apply(attrs_reassign, keep_attrs=True)  

    return q_clim_mean

In [ ]:
clim = ndvi_clim_mean(ds)
print(clim)

In [ ]:
clim = clim.compute()

## Open VP dataset

In [ ]:
vp_clim = xr.open_dataset('../results/VP_test_NDVI_climatology_1987_2010.nc').squeeze()
print(vp_clim)

In [ ]:
# clim.ndvi_clim_mean_DJF.plot(vmin=0, vmax=1.0, cmap='viridis')
# vp_clim.ndvi_clim_meanDJF.plot(vmin=0, vmax=1.0, cmap='viridis')

## Calculate deltas

In [ ]:
diff_DJF = clim.ndvi_clim_mean_DJF - vp_clim.ndvi_clim_meanDJF
diff_MAM = clim.ndvi_clim_mean_MAM - vp_clim.ndvi_clim_meanMAM
diff_JJA = clim.ndvi_clim_mean_JJA - vp_clim.ndvi_clim_meanJJA
diff_SON = clim.ndvi_clim_mean_SON - vp_clim.ndvi_clim_meanSON

### Plot differences, ideally every pixel will be zero

In [ ]:
fig = plt.figure(figsize=(12,12))
ax = fig.subplots(nrows=2, ncols=2, sharex='all')
diff_DJF.plot(ax=ax[0,0], cmap='viridis', vmin=0, vmax=0.5)
diff_MAM.plot(ax=ax[0,1], cmap='viridis', vmin=0, vmax=0.5)
diff_JJA.plot(ax=ax[1,0], cmap='viridis', vmin=0, vmax=0.5)
diff_SON.plot(ax=ax[1,1], cmap='viridis', vmin=0, vmax=0.5)

In [ ]:
#Print statements from VP test

#Input data:
xarray.Dataset>
Dimensions:    (time: 787, x: 356, y: 411)
Coordinates:
  * time       (time) datetime64[ns] 1987-05-29T23:14:05.701791 ... 2010-12-30T23:43:03.718821
  * y          (y) float64 -3.721e+06 -3.721e+06 ... -3.733e+06 -3.733e+06
  * x          (x) float64 1.573e+06 1.574e+06 1.574e+06 ... 1.584e+06 1.584e+06
Data variables:
    nbart_red  (time, y, x) int16 dask.array<chunksize=(1, 250, 250), meta=np.ndarray>
    nbart_nir  (time, y, x) int16 dask.array<chunksize=(1, 250, 250), meta=np.ndarray>
Attributes:
    crs:      EPSG:3577

# NDVI data            
<xarray.Dataset>
Dimensions:  (time: 787, x: 356, y: 411)
Coordinates:
  * time     (time) datetime64[ns] 1987-05-29T23:14:05.701791 ... 2010-12-30T23:43:03.718821
  * y        (y) float64 -3.721e+06 -3.721e+06 ... -3.733e+06 -3.733e+06
  * x        (x) float64 1.573e+06 1.574e+06 1.574e+06 ... 1.584e+06 1.584e+06
Data variables:
    ndvi     (time, y, x) float64 dask.array<chunksize=(1, 250, 250), meta=np.ndarray>
Attributes:
    crs:      EPSG:3577

#Output dataset
<xarray.Dataset>
Dimensions:             (x: 356, y: 411)
Coordinates:
  * y                   (y) float64 -3.721e+06 -3.721e+06 ... -3.733e+06
  * x                   (x) float64 1.573e+06 1.574e+06 ... 1.584e+06 1.584e+06
Data variables:
    ndvi_clim_mean_JFM  (y, x) float64 dask.array<chunksize=(250, 250), meta=np.ndarray>
    ndvi_clim_mean_FMA  (y, x) float64 dask.array<chunksize=(250, 250), meta=np.ndarray>
    ndvi_clim_mean_MAM  (y, x) float64 dask.array<chunksize=(250, 250), meta=np.ndarray>
    ndvi_clim_mean_AMJ  (y, x) float64 dask.array<chunksize=(250, 250), meta=np.ndarray>
    ndvi_clim_mean_MJJ  (y, x) float64 dask.array<chunksize=(250, 250), meta=np.ndarray>
    ndvi_clim_mean_JJA  (y, x) float64 dask.array<chunksize=(250, 250), meta=np.ndarray>
    ndvi_clim_mean_JAS  (y, x) float64 dask.array<chunksize=(250, 250), meta=np.ndarray>
    ndvi_clim_mean_ASO  (y, x) float64 dask.array<chunksize=(250, 250), meta=np.ndarray>
    ndvi_clim_mean_SON  (y, x) float64 dask.array<chunksize=(250, 250), meta=np.ndarray>
    ndvi_clim_mean_OND  (y, x) float64 dask.array<chunksize=(250, 250), meta=np.ndarray>
    ndvi_clim_mean_NDJ  (y, x) float64 dask.array<chunksize=(250, 250), meta=np.ndarray>
    ndvi_clim_mean_DJF  (y, x) float64 dask.array<chunksize=(250, 250), meta=np.ndarray>